In [1]:
# Import statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4 as nc
import xesmf as xe

# Path to data folder
path_to_data = '/home/disk/pna2/aodhan/SurfaceTrendLearning/'
path_to_raw_trends = path_to_data + 'ts_trends.pickle'

# Load dictionary which containes all 19 year trends
TrendDictionary = pd.read_pickle(path_to_raw_trends)

In [2]:
models_for_training_count = 0 # Count of models with at least 10 ensembles
training_maps = []
for model in TrendDictionary.keys():
    if (len(TrendDictionary[model].keys())) >= 11:
        # CMIP6 models must be regridded, below we define input and output grids
        InputGrid = TrendDictionary[model]['grid']
        OutputGrid = {"lon": np.arange(1.25, 358.751, 2.5), "lat": np.arange(-88.75, 88.751, 2.5)}
        regridder = xe.Regridder(InputGrid, OutputGrid, "bilinear")
        models_for_training_count +=1

        # regrid the trend maps for each ensemble member and time period
        EnsembleTrendsArray = []
        for ensemble_name in TrendDictionary[model].keys():
            # data is stored with first element as grid of model
            if ensemble_name == 'grid':
                continue

            ensemble_member = TrendDictionary[model][ensemble_name]

            # return the timeperiods and trend maps in ensemble_member dictionary
            result = ensemble_member.items()
            data = np.array(list(result))
            timeperiods = data[:,0]
            timeperiods = np.array([list(period) for period in timeperiods]) # we will use this as indices for NetCDF
            trend_maps = data[:,1] # these are the indivisual 19 year trend maps

            # Regrid trend map to 2.5 x 2.5 grid
            trend_maps_2p5x2p5 = [regridder(trend_maps[i]) for i in range(0, len(trend_maps))]

            # Return all ensemble trends 
            EnsembleTrendsArray.append(trend_maps_2p5x2p5)
        
        EnsembleTrendsArray = np.array(EnsembleTrendsArray)
        ForcedTrend = np.nanmean(EnsembleTrendsArray, axis=0)
        NaturalTrendsArray = EnsembleTrendsArray - ForcedTrend
        ForcedNaturalTraining = [[NaturalTrendsArray[i], ForcedTrend] for i in range(0, len(NaturalTrendsArray))]

        # Training data will be dumped into NetCDF files
        fileName = path_to_data + model.replace("-", "_") + '_TS_TrendMaps.nc'

        # Create file with dimensions
        ds = nc.Dataset(fileName, 'w', format='NETCDF4')
        ensemble_member = ds.createDimension('ensemble_member', len(ForcedNaturalTraining))
        ForcedNatural = ds.createDimension('ForcedNatural', 2)
        TrendTimePeriod = ds.createDimension('TrendTimePeriod', 30)
        Lat = ds.createDimension('Lat', 72)
        Lon = ds.createDimension('Lon', 144)

        # Add variables to dimensions
        ensemble_member = ds.createVariable('ensemble_member', int, ('ensemble_member',))
        ForcedNatural = ds.createVariable('ForcedNatural', int, ('ForcedNatural',))
        TrendTimePeriod = ds.createVariable('TrendTimePeriod', int, ('TrendTimePeriod',))
        Lat = ds.createVariable('Lat', 'f4', ('Lat',))
        Lon = ds.createVariable('Lon', 'f4', ('Lon',))
        Ts_trends = ds.createVariable('ts_trend', 'f4', ('ensemble_member', 'ForcedNatural', 'TrendTimePeriod', 'Lat', 'Lon'))

        # Assing values to variables
        ensemble_member[:] = np.arange(len(ForcedNaturalTraining)) + 1
        ForcedNatural[:] = [0,1]
        TrendTimePeriod[:] = timeperiods[:,0]
        Lat[:] = np.arange(-88.75, 88.751, 2.5)
        Lon[:] = np.arange(1.25, 358.751, 2.5)
        Ts_trends[:] = ForcedNaturalTraining

        ds.close()



ACCESS-CM2
ACCESS-ESM1-5
AWI-CM-1-1-MR
AWI-ESM-1-1-LR
BCC-CSM2-MR
BCC-ESM1
CAMS-CSM1-0
CAS-ESM2-0
CESM2
CESM2-FV2
CESM2-WACCM
CESM2-WACCM-FV2
CIESM
CMCC-CM2-HR4
CMCC-CM2-SR5
CMCC-ESM2
CNRM-CM6-1
CNRM-CM6-1-HR
CNRM-ESM2-1
CanESM5
CanESM5-CanOE
E3SM-1-0
E3SM-1-1
E3SM-1-1-ECA
EC-Earth3
EC-Earth3-AerChem
EC-Earth3-CC
EC-Earth3-Veg
EC-Earth3-Veg-LR
FGOALS-f3-L
FIO-ESM-2-0
GFDL-CM4
GFDL-ESM4
GISS-E2-1-G
GISS-E2-1-G-CC
GISS-E2-1-H
GISS-E2-2-G
GISS-E2-2-H
GISS-E3-G
HadGEM3-GC31-LL
HadGEM3-GC31-MM
INM-CM4-8
INM-CM5-0
IPSL-CM5A2-INCA
IPSL-CM6A-LR
IPSL-CM6A-LR-INCA
KACE-1-0-G
KIOST-ESM
MCM-UA-1-0
MIROC-ES2H
MIROC-ES2L
MIROC6
MPI-ESM-1-2-HAM
MPI-ESM1-2-HR
MPI-ESM1-2-LR
MRI-ESM2-0
NESM3
NorCPM1
NorESM2-LM
NorESM2-MM
SAM0-UNICON
TaiESM1
UKESM1-0-LL
UKESM1-1-LL


In [3]:
import xarray as xr
thing = xr.open_dataset(path_to_data + 'ACCESS_ESM1_5_TS_TrendMaps.nc')

In [4]:
thing

<xarray.Dataset>
Dimensions:          (ensemble_member: 40, ForcedNatural: 2, TrendTimePeriod: 30, Lat: 72, Lon: 144)
Coordinates:
  * ensemble_member  (ensemble_member) int64 1 2 3 4 5 6 7 ... 35 36 37 38 39 40
  * ForcedNatural    (ForcedNatural) int64 0 1
  * TrendTimePeriod  (TrendTimePeriod) int64 1996 1991 1986 ... 1861 1856 1851
  * Lat              (Lat) float32 -88.75 -86.25 -83.75 ... 83.75 86.25 88.75
  * Lon              (Lon) float32 1.25 3.75 6.25 8.75 ... 353.8 356.2 358.8
Data variables:
    ts_trend         (ensemble_member, ForcedNatural, TrendTimePeriod, Lat, Lon) float32 ...